In [ ]:
import pandas as pd
import re
import yahooquery as yq
import datetime
import numpy as np

In [ ]:
# Leggi il file CSV
df = pd.read_csv('/home/leonardo/Documenti/UNIFI/LAUREA/MATERIALE/hive/tesi28.csv', sep=';')
df


## scarica i valori di borsa per ogni ticker, per ogni anno

In [ ]:
tickers = df['Ticker symbol']
valori_borsa = []
tickers_non_scaricati=[]

# Scarica i dati storici per ciascun ticker e anno
for ticker in tickers:
    yq_ticker = yq.Ticker(ticker+".mi")
    for year in range(2022, 2012, -1):
        # Specifica la data desiderata come "YYYY-MM-DD" (31 dicembre dell'anno specificato)
        datainizio = f"{year}-12-20"
        datafine = f"{year}-12-31"

        # Converti la data desiderata in un oggetto datetime
        datainizio = datetime.datetime.strptime(datainizio, "%Y-%m-%d").date()
        datafine = datetime.datetime.strptime(datafine, "%Y-%m-%d").date()


        try:
            val = yq_ticker.history(start=datainizio, end=datafine)['close'][-1]
        except Exception as e:
            print(e.__cause__)
            val = "NaN"
            tickers_non_scaricati.append(ticker)

        print(ticker, val)
        valori_borsa.append(val)



In [ ]:
print(len(valori_borsa))
print(set(tickers_non_scaricati))

## crea nuovo df
1. prendi info di interesse, moltiplica per ogni anno ogni osservazione 
2. aggiungi gli anni di interesse
3. doppio ciclo for: per ogni riga df originale prendi i primi nove valori e trasponi il risultato, passa ai prossimi nove (secondo ciclo)

#### crea df delle info 

In [ ]:
info = pd.concat([df.iloc[:, 0:5]] * 10)
info.sort_index(inplace=True)
info

#### crea colonna degli anni, aggiungi a info

In [ ]:
anni = [anno for anno in range(2022, 2012, -1)] * 28
info['Anni'] = anni
info

#### df degli indici

In [ ]:

# Rimuovi il carattere '\n' e tutti quelli che seguono dalle intestazioni delle colonne
df.columns = df.columns.str.replace(r'\n.*', '', regex=True)
nomi_colonne = df.columns.unique()[5:]
# prendi df di interesse, serve a semplificare gli indici
indici = df.iloc[:, 5:]

# crea nuovo df
new_df = pd.DataFrame()

col = 0
#per ogni set di 10 valori riga di df originario INDICI
for i in range(0, 140, 10):
    
    indice = []
    #per ogni riga AZIENDA
    for j in range(1,len(indici)+1):
        # prendi i nove valori, trasponili e aggiungili al vettore dell'indice considerato
        valori = indici.iloc[j-1:j, i:i+10].values[0]
        valori = valori.T.tolist()
        
        indice= indice + valori
        
    # aggiungi colonna a new_df con insert
    new_df.insert(col, nomi_colonne[col], indice)
    col += 1

new_df


#### aggiungi anche valori azionari

In [ ]:
# aggiungi i valori della borsa al df info
info['Valori Borsa'] = valori_borsa
info

##### esegui il join dei due dataframe

In [ ]:
dati_puliti = info.join(new_df)
dati_puliti


##### ripulisci i dati, togli le aziende che non hanno valori di brosa per almeno un anno

In [ ]:
tickers_non_scaricati = list(set(tickers_non_scaricati))
maschera = dati_puliti['Ticker symbol'].isin(tickers_non_scaricati)
df_pulito = dati_puliti[~maschera]
df_pulito

### salva il nuovo df 


In [ ]:
df_pulito.to_csv("dati_puliti.csv")